<a href="https://colab.research.google.com/github/YNWA-Algo/IBM-MLAI-CERT-PROJECT3/blob/main/6_M6_2_Pretrained_RNNs_TextClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo">
</center>


# **Recurrent Neural Networks**


Estimated time needed: **45** minutes

A recurrent neural network (RNN) is a type of artificial neural network which uses sequential data or time series data as input. Its typically used for ordinal or temporal problems like language translation, speech recognition, and time series forecasting.

In this lab, we will understand the fundamental building blocks of an RNN. We will train a simple binary text classifier on top of an existing pre-trained module that embeds sentences.


## __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="#Importing-Required-Libraries">Importing Required Libraries</a></li>
            <li><a href="#Defining-Helper-Functions">Defining Helper Functions</a></li>
        </ol>
    </li>
    <li>
        <a href="#RNN-Fundamentals">RNN Fundamentals</a>
        <ol>
            <li><a href="#Vanilla-Recurrent-Neural-Network"> Vanilla Recurrent Neural Network</a></li>
            <li><a href="#Unrolling-in-time-of-a-RNN">Unrolling in time of a RNN</a></li>
            <li><a href="#Training-an-RNN">Training an RNN</a></li>
        </ol>
    </li>
    <li><a href="#Types-of-RNNs">Types of RNNs</a></li>
    <li><a href="#Pre-trained-RNNs">Pre-trained RNNs</a></li>
</ol>


## Objectives

After completing this lab you will be able to:

 - Describe the fundamental building blocks of RNNs.
 - Implement pre-trained RNNs to solve time-series prediction, and forecasting, and text classification tasks


----


## Setup


For this lab, we will be using the following libraries:

*   [`pandas`](https://pandas.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for managing the data.
*   [`numpy`](https://numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for mathematical operations.
*   [`sklearn`](https://scikit-learn.org/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for machine learning and machine-learning-pipeline related functions.
*   [`seaborn`](https://seaborn.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for visualizing the data.
*   [`matplotlib`](https://matplotlib.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for additional plotting tools.


### Installing Required Libraries

The following required libraries are pre-installed in the Skills Network Labs environment. However, if you run these notebook commands in a different Jupyter environment (like Watson Studio or Ananconda), you will need to install these libraries by removing the `#` sign before `!mamba` in the code cell below.


In [2]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
!pip install -q pandas numpy seaborn matplotlib scikit-learn
# Note: If your environment doesn't support "!mamba install", use "!pip install"

The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [3]:
%%capture

!pip install tensorflow_hub
!pip install tensorflow --upgrade
!pip install -q tqdm

In [4]:
!pip install skillsnetwork

### Importing Required Libraries


In [9]:
#import from tensorflow.keras instead of standalone Keras
from tensorflow.keras.preprocessing.text import Tokenizer

In [15]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
print(tf. __version__)
import skillsnetwork
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.losses import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding,Masking,LSTM, GRU, Conv1D, Dropout
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, SimpleRNN
from tensorflow.keras.datasets import reuters
from sklearn.model_selection import train_test_split
from keras.layers import TextVectorization
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
import tensorflow_hub as hub


# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

sns.set_context('notebook')
sns.set_style('white')
np.random.seed(2024)

2.18.0


## Helper Functions


In [16]:
# function to compute the accuracy, precision, recall and F1 score of a model's predictions.
def calculate_results(y_true, y_pred):
    model_accuracy = accuracy_score(y_true, y_pred)
    model_precision, model_recall, model_f1,_ = precision_recall_fscore_support(y_true, y_pred,average="weighted")
    model_results = {"accuracy":model_accuracy,
                     "precision":model_precision,
                     "recall" :model_recall,
                     "f1":model_f1}
    return model_results

## RNN Fundamentals

RNNs fall in the category of neural networks that maintain some kind of **state**. They can process sequential data of arbitrary length. By doing so, they overcome certain limitations faced by classical neural networks. Classical NNs only accept fixed-length vectors as input and output fixed-length vectors. RNNs operate over sequences of vectors. Classical NNs aren't built to consider the sequential nature of some data. RNNs work with sequential data forms like language, video frames, time series, and so on.

The RNN layer uses a for-loop to iterate over the time-steps of a sequence, and maintains an internal state that encodes information about all time-steps that have been observed so far. The Keras RNN API has built-in `keras.layers.RNN` and `keras.layers.LSTM` layers that make it easy to quickly build RNN models.


### Vanilla Recurrent Neural Network

RNNs use these two simple formulas:

$$ \mathbf s_t = \mbox{tanh }(U \mathbf x_t + W \mathbf s_{t-1}) $$

$$ \mathbf y_t = V \mathbf s_t $$

The following plot shows the hyperbolic tan function, `tanh`:

<img src="https://github.com/DataScienceUB/DeepLearningMaster2019/blob/master/images/TanhReal.gif?raw=1" alt="" style="width: 300px;">

#### Terminology:
* $s_t$ current network, or the hidden state
* $\mathbf s_{t-1}$ previous hidden state
* $\mathbf x_t$ current input
* $U, V, W$ matrices that are parameters of the RNN
* $\mathbf y_t$ output at time $t$

These equations say that the current network state or the hidden state, is a function of the previous hidden state and the current input.

### Unrolling in time of a RNN

Given an input sequence, we apply RNN formulas in a recurrent way until we process all input elements. The $U,V,W$ parameters are shared across all recurrent steps. This implies that at each time step, the output is a function of all inputs from previous time steps. The network has a form of memory, encoding information about the time-steps it has seen so far.

Some important observations:
- The initial values for $U,V,W$ as well as for $\mathbf s$ must be provided when training an RNN.
- Hidden state  acts as a memory of the network. It can capture information about the previous steps. It embeds the representation of the sequence.
- We can look at the network's output at every stage or just the final stage.

### Training an RNN

A RNN has a layer for each time step, and its weights are shared across time. It is trained using backpropagation through time, and is done using the following steps:
- The input or the training set is made of several input ($n$-dimensional) sequences $\{\mathbf{X}_i \}$ and corresponding outcomes. Each element of a sequence $\mathbf{x}_j \in \mathbf{X}_i$ is also a vector.
- We use a loss function to measure how well the network's output fits to the expected outcome, such as ground truth.
- We apply an optimization method like stochastic gradient descent or Adam to optimize the loss function
- After the forward pass, gradients of the cost function are propagated backwards through the unrolled network


## Types of RNNs

Predicting the output, $y_t$, at each time step is not always the case. Different RNN architectures can be used to solve different kinds of problems.


|Type|Input|Output|Example problem
|-|-|-|-
|*many-to-many*|An input sequence|An output sequence|Part of Speech (POS) tagging
|*many-to-one*|An input sequence|Value of output sequence for last timestep|Text classification: positive tweet or negative?
|*one-to-many*|Single value of input sequence|An output sequence| Given an input image, predict sequence data


## Pre-trained RNNs


In this section, we will be experimenting with existing RNNs. We will use the NLP disaster dataset. The dataset contains a `test.csv` and a `train.csv` each of which have the following information:

* The text of a tweet
* A keyword from that tweet (although this may be blank!)
* The location the tweet was sent from (may also be blank)

Our task is to predict whether a given tweet is about a real disaster or not. If so, predict a 1. If not, predict a 0.


Let us start by downloading and unzipping the dataset.


In [17]:
await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module4/L1/nlp_disaster.zip",overwrite=True)


  0%|          | 0/3 [00:00<?, ?it/s]

Saved to '.'


Now we will read in the train dataset. Here we use `frac=1` so all rows in the training dataset are returned in a random order. We also set a random state to ensure reproducibility of results.


In [18]:
train_df = pd.read_csv("train.csv")
# shuffle the dataset
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


We will use 90% of the entire labelled dataset for training, and 10% of it for testing purposes.


In [19]:
# split the data into 90% training and 10% testing
X_train, X_test, y_train, y_test = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                    train_df_shuffled["target"].to_numpy(),
                                                    test_size = 0.1,
                                                    random_state=42)
X_train.shape, y_train.shape

((6851,), (6851,))

In [ ]:
X_train[0:5]

`TextVectorization` is a preprocessing layer which maps text features to integer sequences. We also specify `lower_and_strip_punctuation` as the standardization method to apply to the input text. The text will be lowercased and all punctuation removed. Next we split on the whitespace, and pass `None` to `ngrams` so no ngrams are created.



In [20]:
text_vectorizer = TextVectorization(max_tokens=None,
                                    #remove punctuation and make letters lowercase
                                    standardize="lower_and_strip_punctuation",
                                    #whitespace delimiter
                                    split="whitespace",
                                    #dont group anything, every token alone
                                    ngrams = None,
                                    output_mode ="int",
                                    #length of each sentence == length of largest sentence
                                    output_sequence_length=None
                                    )


In [21]:
# define hyperparameters

# number of words in the vocabulary
max_vocab_length = 10000
# tweet average length
max_length = 15

Below we define an `Embedding` layer with a vocabulary of 10,000, a vector space of 128 dimensions in which words will be embedded, and input documents that have 15 words each.


In [22]:
embedding = layers.Embedding(input_dim= max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

The `hub.KerasLayer` wraps a SavedModel (or a legacy TF1 Hub format) as a Keras Layer. The `universal-sentence-encoder` is an encoder of greater-than-word length text trained on a variety of data. It can be used for text classification, semantic similarity, clustering, and other natural language tasks.

> We can train a simple binary text classifier on top of any TF-Hub module that can embed sentences. The Universal Sentence Encoder was partially trained with custom text classification tasks in mind. These kinds of classifiers can be trained to perform a wide variety of classification tasks often with a very small amount of labeled examples.

More on this is found in the Tensorflow Hub [documentation](https://tfhub.dev/google/universal-sentence-encoder/4)


In [23]:
encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                               input_shape=[],
                               dtype = tf.string,
                               trainable=False,
                               name="pretrained")

The `encoder_layer` will take as input variable length English text and the output is a 512 dimensional vector.


We will add a Dense layer with unit 1 to create a simple binary text classifier on top of any TF-Hub module. Next, we will compile and fit it using 20 epochs.


In [24]:
model = tf.keras.Sequential([
                             encoder_layer,
                             layers.Dense(1,activation="sigmoid")], name="model_pretrained")
model.compile(loss="binary_crossentropy",
                     optimizer="adam",
                     metrics=["accuracy"])

model.fit(x=X_train,
              y=y_train,
              epochs=20,
              validation_data=(X_test,y_test))

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7d9b2ae6cb50> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [26]:
calculate_results(y_true=y_test,
                  y_pred=tf.squeeze(tf.round(model.predict(X_test))))

NameError: name 'model' is not defined

The model is able to predict the tweet class with a fairly high accuracy.


## Authors


[Kopal Garg](https://www.linkedin.com/in/gargkopal/)


Kopal is a Masters student in Computer Science at the University of Toronto.


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2022-07-18|0.1|Kopal|Create Lab|
|2022-08-30|0.1|Steve Hord|QA pass edits|


Copyright © 2022 IBM Corporation. All rights reserved.
